To begin, we first need to import the necessary libraries:

In [1]:
import geopandas as gpd
import numpy as np
import shapely as shp
import pyomo.environ as pyo

Next, we need to read the file and inspect the data. We check for:
1. The number of polygons
2. The minimum, average, and maximum areas of each polygon
3. The range of *carbon_store* and *cost* properties, defined as the difference in max and min values of each property

In [27]:
parcels = gpd.read_file("land_parcels.shp").to_crs('EPSG:3347')
parcels['area'] = parcels.area / ((10 ** 3) ** 2)

# Print minimum area
print("Number of polygons:", len(parcels['parcel_id']))

print("Minimum area (km^2):", min(parcels['area']))
print("Average area (km^2):", np.average(parcels['area']))
print("Maximum area (km^2):", max(parcels['area']))

print("carbon_store range:", max(parcels['carbon_sto']) - min(parcels['carbon_sto']))
print("cost range:", max(parcels['cost']) - min(parcels['cost']))

Number of polygons: 100
Minimum area (km^2): 37.37824723691204
Average area (km^2): 80.54003288468247
Maximum area (km^2): 132.0955428630858
carbon_store range: 89.82436510414392
cost range: 4457.482333726783


We then check for and remove outliers in the data. Outliers are defined as any parcel with an area less than three times the interquartile range of the parcels' areas.

In [5]:
q1 = np.percentile(parcels['area'], 25)
q3 = np.percentile(parcels['area'], 75)
iqr = q3 - q1
range_min = q1 - 3 * iqr

count = 0
for index, row in parcels.iterrows():
    if row['area'] < range_min:
        parcels.drop(index, inplace=True)
        count += 1

print(count, "outliers detected and removed")

0 outliers detected and removed


Next, we construct the adjacency list. From visually inspecting the data, each polygon is made of four points, with the fifth point indicating the end of the polygon. Adjacent edges occur when two polygons share at least one pair of consecutive coordinates. Note that one polygon's pair could be in the same order as the second polygon's pair or in reverse order.

The following code creates the adjacency list by checking whether each polygon shares pairs of consecutive coordinates with every remaining polygon. The parcel_id of the polygon being checked is added to the remaining polygon's sublist in the adjacency list.

In [7]:
def adjacency_build_helper(coords1, coords2):
    match = False
    curr_idx = 0
    while (not match) and curr_idx < 4:
        current = (coords1[curr_idx], coords1[curr_idx + 1])
        for nextIdx in range(4):
            if (np.array_equal(current[0], coords2[nextIdx]) and np.array_equal(current[1], coords2[nextIdx + 1]) or
                    np.array_equal(current[0], coords2[nextIdx + 1]) and np.array_equal(current[1], coords2[nextIdx])):
                match = True
                break
        curr_idx += 1
    return match

coordinates = [shp.get_coordinates(geo) for geo in parcels['geometry']]
adjacency_list = dict()
for i in range(100 - count):
    adjacents = []
    for j in range(99 - count):
        if adjacency_build_helper(coordinates[i], coordinates[(i + j + 1) % 100]): adjacents.append((i + j + 1) % 100)
    adjacency_list[i] = adjacents

Lastly, we use Pyomo to create and solve the optimization problem. We construct the necessary dictionaries for Pyomo to use as variables (the numbers Pyomo modifies to optimize the problem) and parameters (the numbers Pyomo leaves alone as constants).

The costs, areas, carbon stores, and adjacency list are set as parameters while the variables are set as whether the parcels are being selected (0 for not selected, 1 as selected).

In [9]:
full_cost = sum(parcels['cost'])
full_area = sum(parcels['area'])
parcels_dict = parcels.to_dict()

parcel_costs_init = parcels_dict['cost']
parcel_areas_init = parcels_dict['area']
parcel_carbon_stores_init = parcels_dict['carbon_sto']
parcel_selection_init = dict([(parcel_id, 0) for parcel_id in parcel_costs_init.keys()])

model = pyo.ConcreteModel()

model.parcels_id = pyo.Set(initialize=parcel_selection_init.keys())
model.parcels_area = pyo.Param(model.parcels_id, initialize=parcel_areas_init)
model.parcels_cost = pyo.Param(model.parcels_id, initialize=parcel_costs_init)
model.carbon_stores = pyo.Param(model.parcels_id, initialize=parcel_carbon_stores_init)
model.adjacency_list = pyo.Param(model.parcels_id, initialize=adjacency_list)

model.selection = pyo.Var(model.parcels_id, domain=pyo.Binary, initialize=parcel_selection_init)

of 'Any'. The default domain for Param objects is 'Any'.  However, we will be
changing that default to 'Reals' in the future.  If you really intend the
domain of this Paramto be 'Any', you can suppress this warning by explicitly
specifying 'within=Any' to the Param constructor.  (deprecated in 5.6.9, will
be removed in (or after) 6.0) (called from C:\Users\danan\anaconda3\Lib\site-
packages\pyomo\core\base\indexed_component.py:718)


As mentioned in the problem document, the objective is to find the polygons that produce the maximum carbon store under specific constraints. We define this in Pyomo below:

In [19]:
model.objective = pyo.Objective(expr=sum(model.carbon_stores[parcel_id] * model.selection[parcel_id]
                                         for parcel_id in model.parcels_id),
                                sense=pyo.maximize)

'pyomo.core.base.objective.ScalarObjective'>) on block unknown with a new
Component (type=<class 'pyomo.core.base.objective.ScalarObjective'>). This is
usually indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


Next, we define the constraints as mentioned in the project manual.

In [21]:
# Constraint 1: Cost of all selected polygons is less than 0.5 total cost of remaining parcels after filtering
model.constraint_budget = pyo.Constraint(
    expr=sum([model.selection[id] * model.parcels_cost[id] for id in model.parcels_id]) <= 0.5 * full_cost)


# Constraint 2: No two selected polygons are adjacent to each other
def constraint_2(model):
    any_adjacency = pyo.Constraint.Feasible
    list_id = 0

    while list_id < len(model.parcels_id) and not any_adjacency:
        if pyo.value(model.selection[list_id]) == 1:
            for parcel_id in model.parcels_id:
                if pyo.value(model.selection[parcel_id]) == 1 and parcel_id in model.adjacency_list[list_id]:
                    any_adjacency = pyo.Constraint.Infeasible
                    break
        list_id += 1

    return any_adjacency
model.constraint_adjacency = pyo.Constraint(expr=constraint_2(model))

# (Optional) Constraint 3: Total area must be at least 25% of the total area of remaining parcels after filtering
# model.constraint_area = pyo.Constraint(
#     expr=sum([model.selection[id] * model.parcels_area[id] for id in model.parcels_id]) >= 0.25 * full_area)

(type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
(type=<class 'pyomo.core.base.constraint.ScalarConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().


Lastly, we define the solver used by Pyomo as our GLPK installation, solve the model, and output the polygons used, their total cost, and total ``carbon_store``.

In [29]:
solver = pyo.SolverFactory('glpk', executable="C:\\glpk-4.65\\w64\\glpsol")
results = solver.solve(model)

print("Polygons:", end=" ")
final_cost = 0
for i, j in model.selection.items():
    if pyo.value(j) == 1:
        print(i, end=" ")
        final_cost += pyo.value(model.parcels_cost[i])
print("\nTotal cost", final_cost)

print("Total carbon_store:" , pyo.value(model.objective))

Polygons: 3 6 9 10 11 13 14 16 19 20 22 23 25 26 27 29 30 31 33 34 35 36 39 40 43 44 45 46 48 49 52 53 54 55 56 57 59 60 64 65 66 67 69 70 71 73 74 77 78 79 81 83 84 85 86 88 91 92 93 96 99 
Total cost 140757.06548124866
Total carbon_store: 4350.825955966731


# Report
## Approach
- No AI code was used, but several online resources were consulted for aid.
## Methodology
- Algorithms were brute force loops that iterated over every parcel until final output was constructed:
  - Adjacency list was constructed by checking every pair of consecutive coordinates in one polygon with every pair of consecutive coordinates in every other polygon.
## Challenges
- Due to time constraints in real life, I was only able to accomplish this project during the weekend.
- Learning about each library; I wanted to learn about each library on my own and understand them without the use of generative AI. Solution was to consult tutorials and online forums while inferring how each snippet of code ran and operated.
- Troubleshooting code errors.
## Key Results and Analysis
- Some polygons outputted are adjacent to each other (e.g., 10, 20); can be inspected visually using Google Earth Pro.
- Unable to plot results. List of polygons are given when code is run.
- Given results:
  - Total cost: 140757.06548124866
  - Total carbon_store: 4350.825955966731
## Analysis
- Further revision required due to errors.
  - Unknown error with adjacency constraint in model declaration; will need further revision.
- Further optimization required.
  - Algorithms were brute-force, with high time complexities ($O(n^4)$ for adjacency list construction). Deeper knowledge of algorithms would assist with improving time complexities.